In [1]:

import pandas as pd

ratings = pd.read_csv('/project_data/data_asset/ratings.csv')
ratings.head()



,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [2]:
ratings['userId'].nunique()

162541

In [3]:
ratings.shape

(25000095, 4)

In [4]:

movies = pd.read_csv('/project_data/data_asset/movies.csv')
movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_sub = ratings[ratings['userId']<=1500]

In [6]:
ratings_pivot= ratings_sub.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
ratings_pivot.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,205557,205583,205967,206272,206499,206523,206805,206861,207309,208002
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
import numpy as np
R =ratings_pivot.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [8]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [9]:
sigma = np.diag(sigma)

In [10]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [11]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = ratings_pivot.columns)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,205557,205583,205967,206272,206499,206523,206805,206861,207309,208002
0,0.060143,-0.090199,-0.019365,-0.102161,-0.024189,0.093606,0.104031,-0.033133,-0.094126,-0.017477,...,-0.001913,-0.008182,-0.009852,0.009258,0.019363,-0.002630,0.001432,-0.006512,-0.001298,-0.000895
1,3.594620,0.860223,0.189797,-0.124894,-0.048699,0.344240,0.422066,0.101123,-0.124780,0.835904,...,0.028233,0.050114,0.058619,-0.002757,-0.004994,0.009844,-0.007667,0.041610,-0.024164,-0.017411
2,2.595845,1.109090,-0.299525,-0.035958,-0.144998,-0.192789,0.181123,-0.066710,-0.343011,-0.681663,...,-0.016078,-0.043796,-0.048598,0.012329,-0.005316,0.045803,-0.009475,-0.038993,-0.018011,-0.023518
3,3.684102,-0.081054,-0.329966,-0.010181,-0.142839,0.002766,-0.061583,-0.009142,0.013481,0.186382,...,-0.029382,0.105374,0.123287,0.043561,0.044039,0.048683,-0.008958,0.087462,0.037612,0.044470
4,4.525617,0.382887,1.547149,0.171984,1.374634,1.698826,1.402319,0.021900,0.310801,0.902013,...,-0.031906,-0.027937,-0.032151,-0.003589,-0.016527,0.008364,-0.002461,-0.023724,-0.013100,-0.013295


In [12]:

def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [62]:
already_rated, predictions = recommend_movies(preds, 1, movies, ratings, 10)

User 1 has already rated 70 movies.
Recommending highest 10 predicted ratings movies not already rated.


In [17]:
# Top 20 movies that User 1310 has rated 
already_rated.head(20)

,userId,movieId,rating,timestamp,title,genres
11,1300,223,5.0,845585708,Clerks (1994),Comedy
49,1300,778,5.0,845586056,Trainspotting (1996),Comedy|Crime|Drama
14,1300,282,5.0,845585518,Nell (1994),Drama
24,1300,356,5.0,845585325,Forrest Gump (1994),Comedy|Drama|Romance|War
48,1300,724,5.0,845586506,"Craft, The (1996)",Drama|Fantasy|Horror|Thriller
21,1300,345,5.0,845585827,"Adventures of Priscilla, Queen of the Desert, ...",Comedy|Drama
50,1300,835,4.0,845587275,Foxfire (1996),Drama
46,1300,593,4.0,845585300,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
43,1300,589,4.0,845585382,Terminator 2: Judgment Day (1991),Action|Sci-Fi
42,1300,535,4.0,845585979,Short Cuts (1993),Drama


In [118]:
df = pd.DataFrame()
for i in range(1,1500):
    already_rated, predictions = recommend_movies(preds, i, movies, ratings, 10)
    tmp_df = pd.DataFrame()
    predictions['userId']=i
    predictions['rank']= predictions.reset_index().index + 1
    pivot_pred = pd.pivot_table(predictions, values='title', index='userId', columns='rank', aggfunc='first').reset_index()
    df = df.append(pivot_pred)  

User 1 has already rated 70 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 2 has already rated 184 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 3 has already rated 656 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 4 has already rated 242 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 5 has already rated 101 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 6 has already rated 26 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 7 has already rated 25 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 8 has already rated 155 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 9 has already rated 178 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 10 has already rated 53 movies.
Recommending highest 

User 79 has already rated 22 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 80 has already rated 517 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 81 has already rated 23 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 82 has already rated 170 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 83 has already rated 28 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 84 has already rated 314 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 85 has already rated 68 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 86 has already rated 148 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 87 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 88 has already rated 21 movies.
Recommending h

User 157 has already rated 25 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 158 has already rated 78 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 159 has already rated 230 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 160 has already rated 96 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 161 has already rated 87 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 162 has already rated 101 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 163 has already rated 52 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 164 has already rated 148 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 165 has already rated 65 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 166 has already rated 611 movies.
Reco

User 235 has already rated 146 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 236 has already rated 215 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 237 has already rated 70 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 238 has already rated 61 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 239 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 240 has already rated 196 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 241 has already rated 20 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 242 has already rated 160 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 243 has already rated 660 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 244 has already rated 111 movies.
Re

User 313 has already rated 147 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 314 has already rated 48 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 315 has already rated 51 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 316 has already rated 40 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 317 has already rated 30 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 318 has already rated 430 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 319 has already rated 162 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 320 has already rated 79 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 321 has already rated 791 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 322 has already rated 338 movies.
Rec

...

User 1173 has already rated 449 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1174 has already rated 43 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1175 has already rated 157 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1176 has already rated 96 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1177 has already rated 199 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1178 has already rated 117 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1179 has already rated 45 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1180 has already rated 43 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1181 has already rated 26 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1182 has already rated 29 mo

User 1250 has already rated 33 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1251 has already rated 20 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1252 has already rated 108 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1253 has already rated 108 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1254 has already rated 355 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1255 has already rated 45 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1256 has already rated 29 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1257 has already rated 22 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1258 has already rated 32 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1259 has already rated 163 mo

User 1328 has already rated 112 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1329 has already rated 354 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1330 has already rated 226 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1331 has already rated 83 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1332 has already rated 25 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1333 has already rated 51 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1334 has already rated 204 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1335 has already rated 180 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1336 has already rated 169 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1337 has already rated 285

User 1406 has already rated 22 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1407 has already rated 128 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1408 has already rated 42 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1409 has already rated 745 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1410 has already rated 21 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1411 has already rated 157 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1412 has already rated 28 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1413 has already rated 40 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1414 has already rated 107 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1415 has already rated 55 mo

User 1484 has already rated 869 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1485 has already rated 32 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1486 has already rated 149 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1487 has already rated 32 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1488 has already rated 312 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1489 has already rated 40 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1490 has already rated 70 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1491 has already rated 79 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1492 has already rated 187 movies.
Recommending highest 10 predicted ratings movies not already rated.
User 1493 has already rated 32 mo

In [119]:
df = df.reset_index().drop('index', axis =1)

In [120]:
df = df.add_prefix('rec_')

In [121]:
df

rank,rec_userId,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10
0,1,"Lord of the Rings: The Fellowship of the Ring,...","Lord of the Rings: The Return of the King, The...",Memento (2000),Kill Bill: Vol. 1 (2003),Fight Club (1999),Kill Bill: Vol. 2 (2004),Donnie Darko (2001),"Matrix, The (1999)",Léon: The Professional (a.k.a. The Professiona...,"Monsters, Inc. (2001)"
1,2,Finding Nemo (2003),Good Will Hunting (1997),Dances with Wolves (1990),"Green Mile, The (1999)",Beauty and the Beast (1991),Die Hard (1988),Star Wars: Episode II - Attack of the Clones (...,Batman (1989),Star Wars: Episode I - The Phantom Menace (1999),"Terminator, The (1984)"
2,3,Donnie Darko (2001),Black Hawk Down (2001),Kingsman: The Secret Service (2015),One Flew Over the Cuckoo's Nest (1975),Men in Black (a.k.a. MIB) (1997),A.I. Artificial Intelligence (2001),National Treasure (2004),Deadpool (2016),Harry Potter and the Sorcerer's Stone (a.k.a. ...,Brazil (1985)
3,4,Iron Man (2008),Toy Story 3 (2010),Star Trek (2009),Zombieland (2009),Shrek (2001),"Terminator, The (1984)",Inside Out (2015),Watchmen (2009),The Imitation Game (2014),Captain America: Civil War (2016)
4,5,"Shawshank Redemption, The (1994)",Mr. Holland's Opus (1995),Jurassic Park (1993),True Lies (1994),Mission: Impossible (1996),Jerry Maguire (1996),Leaving Las Vegas (1995),"Truth About Cats & Dogs, The (1996)","Time to Kill, A (1996)",Raiders of the Lost Ark (Indiana Jones and the...
...,...,...,...,...,...,...,...,...,...,...,...
1494,1495,Seven (a.k.a. Se7en) (1995),Terminator 2: Judgment Day (1991),"Godfather, The (1972)","Godfather: Part II, The (1974)","Shawshank Redemption, The (1994)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Braveheart (1995),American Beauty (1999),Fight Club (1999),Heat (1995)
1495,1496,WALL·E (2008),Finding Nemo (2003),Ratatouille (2007),"Incredibles, The (2004)",Inception (2010),Toy Story 3 (2010),Guardians of the Galaxy (2014),"Avengers, The (2012)",Toy Story 2 (1999),Harry Potter and the Deathly Hallows: Part 2 (...
1496,1497,Dances with Wolves (1990),Philadelphia (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Seven (a.k.a. Se7en) (1995),Crimson Tide (1995),True Lies (1994),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"Net, The (1995)",Interview with the Vampire: The Vampire Chroni...
1497,1498,Dead Man Walking (1995),True Romance (1993),"Usual Suspects, The (1995)",Clerks (1994),"American President, The (1995)",Ransom (1996),Fargo (1996),What's Eating Gilbert Grape (1993),Nell (1994),"River Wild, The (1994)"


In [122]:
user_fav3 = ratings_sub.sort_values(by=['userId', 'rating'], ascending =[True, False])

In [123]:
user_fav3_gb = user_fav3.groupby('userId').head(3).reset_index(drop=True)

In [124]:
user_fav3_gb = user_fav3_gb.merge(movies, on='movieId', how='left')

In [125]:
user_fav3_gb .head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
3,2,110,5.0,1141416589,Braveheart (1995),Action|Drama|War
4,2,260,5.0,1141417172,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


In [126]:
user_fav3_gb['rank_old'] = user_fav3_gb.groupby('userId').cumcount()+1

In [127]:
user_fav3_gb

,userId,movieId,rating,timestamp,title,genres,rank_old
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1
1,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,2
2,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War,3
3,2,110,5.0,1141416589,Braveheart (1995),Action|Drama|War,1
4,2,260,5.0,1141417172,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,2
...,...,...,...,...,...,...,...
4495,1499,318,5.0,1079735610,"Shawshank Redemption, The (1994)",Crime|Drama,2
4496,1499,471,5.0,1079735325,"Hudsucker Proxy, The (1994)",Comedy,3
4497,1500,356,5.0,1554779548,Forrest Gump (1994),Comedy|Drama|Romance|War,1
4498,1500,527,5.0,1554779513,Schindler's List (1993),Drama|War,2


In [128]:
user_fav3_gb_pivot = pd.pivot_table(user_fav3_gb, index='userId', columns='rank_old', values='title', aggfunc='first').reset_index()

In [129]:
user_fav3_gb_pivot = user_fav3_gb_pivot.reset_index().drop('index', axis =1)

In [130]:
user_fav3_gb_pivot = user_fav3_gb_pivot.add_prefix('oldrank_')

In [131]:
user_fav3_gb_pivot

rank_old,oldrank_userId,oldrank_1,oldrank_2,oldrank_3
0,1,Pulp Fiction (1994),Three Colors: Blue (Trois couleurs: Bleu) (1993),Underground (1995)
1,2,Braveheart (1995),Star Wars: Episode IV - A New Hope (1977),"Shawshank Redemption, The (1994)"
2,3,"Usual Suspects, The (1995)",Before the Rain (Pred dozhdot) (1994),Léon: The Professional (a.k.a. The Professiona...
3,4,Spirited Away (Sen to Chihiro no kamikakushi) ...,"Incredibles, The (2004)",Lucky Number Slevin (2006)
4,5,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Dead Man Walking (1995),Seven (a.k.a. Se7en) (1995)
...,...,...,...,...
1495,1496,Spirited Away (Sen to Chihiro no kamikakushi) ...,Up (2009),Inside Out (2015)
1496,1497,Sense and Sensibility (1995),Clueless (1995),"Usual Suspects, The (1995)"
1497,1498,Father of the Bride Part II (1995),GoldenEye (1995),Kids (1995)
1498,1499,Pulp Fiction (1994),"Shawshank Redemption, The (1994)","Hudsucker Proxy, The (1994)"


In [133]:
df = df.merge(user_fav3_gb_pivot, how='left', left_on='rec_userId', right_on='oldrank_userId')

In [134]:
df = df.rename(columns= {'rec_userId':'userId'}).drop('oldrank_userId', axis=1)

In [136]:
#df.to_csv('/project_data/data_asset/pred_sample.csv')

In [137]:
from project_lib import Project

In [138]:
project = Project.access()

In [140]:
project.save_data("pred.csv", df.to_csv())

{'file_name': 'pred.csv',
 'message': 'File saved to project storage.',
 'asset_id': 'b2a741ae-8f76-4aa2-b7ce-99fd0009d446'}